# KAFKA PRODUCER

In [12]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
import tweepy
import os
import json
from kafka import KafkaProducer

ACCESS_TOKEN="1052919765126316033-eATAf2skyDxIomK4GxYaZ7zj5Th6BX"
ACCESS_TOKEN_SECRET="fMFNeiSTpMu4kJuZx0vXYgKeizRGv8CZZQaZ2LbY576lH"
CONSUMER_KEY="0wjVKNcrlxQIlf0UKVNwpwShq"
CONSUMER_SECRET="mUoz9LP810L2bkJzfpM1e8TgK2wLmip3kc7nORjvAbwwmtKxid"

#SEARCH_KEYWORD = str(input("Please enter a keyword to filter\n"))

producer = KafkaProducer(bootstrap_servers=['kafka:9092'])

def filter_tweet(tweet):
    data = json.loads(tweet)
    
    if 'delete' in data:
        return False
    if 'timestamp_ms' not in data or 'entities' not in data:
        return False      
    if len(data['entities']['hashtags']) <= 0:
        return False
        
    data = {"timestamp": data['timestamp_ms'], "hashtags": [i["text"] for i in data['entities']['hashtags']]}
    print(data)
    data = json.dumps(data)
    
    return data


class StdOutListener(StreamListener):
    def __init__(self):
        self.count=0

    def on_data(self, data):
        data = filter_tweet(data)
        if data != False:
            producer.send("tweets", data.encode('utf-8'))  
         
        self.count+=1 
        print(self.count)
        return True
    def on_error(self, status):
        print(status)
        
    def on_exception(self, exception):
        print(exception)
        return

myStreamListener = StdOutListener()
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
myStream = tweepy.Stream(auth = auth, listener=myStreamListener)
#myStream.filter(track=[SEARCH_KEYWORD])
myStream.sample(languages=["en"])

print(f"Starting tweet streams")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
{'timestamp': '1624619325657', 'hashtags': ['btsdiamond', 'BtsDiscography']}
22
23
{'timestamp': '1624619325666', 'hashtags': ['OMEGA_X', '오메가엑스', '재한', '한겸', '예찬', '주간아', '주간아이돌']}
24
25
{'timestamp': '1624619325661', 'hashtags': ['บอกฉันByMew']}
26
27
{'timestamp': '1624619325660', 'hashtags': ['艦これ版深夜の真剣お絵描き60分一本勝負', '艦これ版真剣お絵描き60分一本勝負_20210623', '艦これ', '峯雲']}
28
29
30
31
32
33
34
35
36
37
38
39
40
41
{'timestamp': '1624619325660', 'hashtags': ['colors', 'alien', 'art']}
42
43
44
45
46
47
48
49
{'timestamp': '1624619325662', 'hashtags': ['Transport', 'Africa', 'Kenya', 'startups']}
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
{'timestamp': '1624619326665', 'hashtags': ['プロセカ生放送']}
65
66
{'timestamp': '1624619326665', 'hashtags': ['BamBam', 'BamBam_ChangminsFreehug']}
67
68
69
{'timestamp': '1624619326657', 'hashtags': ['정한', 'JEONGHAN']}
70
71
{'timestamp': '1624619326663', 'hashtags': ['BurgerKingLixo']}
72
73
74
75
76
77
78
79


{'timestamp': '1624619334660', 'hashtags': ['紫陽花', 'hydrangea']}
585
586
587
588
589
590
{'timestamp': '1624619335665', 'hashtags': ['プロセカ生放送']}
591
{'timestamp': '1624619335658', 'hashtags': ['まだアプデしてないの', '料理男子', '西畑大吾', 'なにわ男子', 'アプデ男子', 'アプデ男子集', 'テレビ朝日']}
592
{'timestamp': '1624619335662', 'hashtags': ['プロセカ生放送']}
593
594
595
596
597
{'timestamp': '1624619335663', 'hashtags': ['ポメラニアン', '小トトロ']}
598
{'timestamp': '1624619335662', 'hashtags': ['บาสเด็กอ้วนที่แท้จริง']}
599
{'timestamp': '1624619335661', 'hashtags': ['minne']}
600
601
602
603
604
605
{'timestamp': '1624619335661', 'hashtags': ['باافتخار_ایرانی']}
606
{'timestamp': '1624619335664', 'hashtags': ['뮤직케이_정산내역_공개해', '김수찬_정산받고_안전이별해']}
607
{'timestamp': '1624619335664', 'hashtags': ['GreatestGuru_InTheWorld']}
608
609
{'timestamp': '1624619335663', 'hashtags': ['SaveProdigalSon', 'ModeratelyScathed']}
610
611
612
613
{'timestamp': '1624619335665', 'hashtags': ['NiziU']}
614
615
616
617
618
{'timestamp': '1624619335665', 'h

1081
1082
1083
1084
1085
1086
1087
1088
{'timestamp': '1624619343658', 'hashtags': ['NiziU_Scout', '金曜日は虹スカ']}
1089
1090
{'timestamp': '1624619343657', 'hashtags': ['ครั้งหนึ่งที่รักtser']}
1091
1092
{'timestamp': '1624619343665', 'hashtags': ['JusticeForAtul']}
1093
1094
1095
1096
1097
{'timestamp': '1624619343662', 'hashtags': ['뮤직뱅크']}
1098
{'timestamp': '1624619343663', 'hashtags': ['まほおそファミリー']}
1099
1100
1101
1102
1103
1104
{'timestamp': '1624619343666', 'hashtags': ['WhyModiHatesFarmers', 'FarmersProtest']}
1105
1106
1107
1108
{'timestamp': '1624619343660', 'hashtags': ['PremiosMTVMIAW', 'MTVLAKPOPROSE', 'ROSÉ', '로제']}
1109
1110
1111
1112
{'timestamp': '1624619343659', 'hashtags': ['Decentralisation4Afghanistan']}
1113
1114
1115
1116
1117
1118
1119
1120
1121
1122
1123
1124
1125
1126
1127
1128
1129
1130
1131
{'timestamp': '1624619343663', 'hashtags': ['ADDBiblicalAnswers', 'LearnFromTheBible']}
1132
1133
1134
1135
1136
1137
1138
1139
1140
1141
1142
{'timestamp': '1624619344664', 

KeyboardInterrupt: 